In [ ]:
# 09_recommendation.py

import pandas as pd
from pathlib import Path

# 경로 설정
OUTPUT_DIR = Path("../output")

#  데이터 불러오기
segments_df = pd.read_csv(OUTPUT_DIR / "posts_segmented.csv")   # segment별 정보
topics_df = pd.read_csv(OUTPUT_DIR / "topics_performance.csv")  # topic × score
ab_results_df = pd.read_csv(OUTPUT_DIR / "ab_test_results.csv") # AB test 결과


In [ ]:
#  segment별 추천 전략 작성
def generate_segment_recommendations(segments_df, topics_df):
    recommendations = []

    for segment in segments_df['segment'].unique():
        seg_posts = segments_df[segments_df['segment'] == segment]
        
        # 해당 segment에서 가장 높은 평균 점수를 가진 topic 선택
        top_topic = seg_posts.groupby('topic')['score'].mean().idxmax()
        
        # 대표 게시물 추출
        top_posts = seg_posts[seg_posts['topic'] == top_topic].sort_values('score', ascending=False).head(3)
        top_post_titles = top_posts['title'].tolist()
        
        recommendations.append({
            'segment': segment,
            'recommended_topic': top_topic,
            'top_posts': top_post_titles
        })
    
    return pd.DataFrame(recommendations)

segment_recommendations = generate_segment_recommendations(segments_df, topics_df)


In [ ]:

# AB 테스트 기반 추천 필터링
def refine_recommendations_with_ab(segment_recommendations, ab_results_df):
    # 예시: positive effect만 남기고 negative는 제거
    positive_ab = ab_results_df[ab_results_df['effect_direction'] == 'positive']
    
    refined = segment_recommendations[segment_recommendations['recommended_topic'].isin(positive_ab['topic'])]
    return refined

final_recommendations = refine_recommendations_with_ab(segment_recommendations, ab_results_df)

#  결과 저장
final_recommendations.to_csv(OUTPUT_DIR / "recommendations_final.csv", index=False)
print(" Recommendations saved to recommendations_final.csv")
